In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

In [3]:
df=pd.read_csv("/kaggle/input/customer-purchasing-behaviors/Customer Purchasing Behaviors.csv")

In [4]:
df=pd.get_dummies(df).astype(int)

In [14]:
df.isnull().sum()

user_id               0
age                   0
annual_income         0
purchase_amount       0
loyalty_score         0
purchase_frequency    0
region_East           0
region_North          0
region_South          0
region_West           0
dtype: int64

In [6]:
X=df.drop(columns=["age"])
Y=df["age"]
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [47]:
# Y_train

184    43
224    52
194    52
95     41
227    50
       ..
106    41
14     49
92     51
179    55
102    46
Name: age, Length: 190, dtype: int64

In [7]:
from keras.layers import Layer,Dense,Flatten
from keras.models import Sequential

In [8]:
!pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 65.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.6 M

In [9]:
import cirq

In [15]:
X_train.isnull().sum()

user_id               0
annual_income         0
purchase_amount       0
loyalty_score         0
purchase_frequency    0
region_East           0
region_North          0
region_South          0
region_West           0
dtype: int64

In [42]:
from keras.layers import Layer,LeakyReLU,BatchNormalization
from keras.models import Model,Sequential
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [43]:
early_stopping=EarlyStopping(
monitor="val_loss",
patience=5,
verbose=1
)

In [45]:
def quantum_circuit(data):
    qubit=[cirq.GridQubit(0,i) for i in range(4)]
    circuit=cirq.Circuit()
    for i,value in enumerate(data):
        new_data=i%(2*np.pi)
        circuit.append(cirq.rx(new_data).on(qubit[i%len(qubit)]))
        if i%len(qubit)>=0:
            circuit.append(cirq.CNOT(qubit[i%len(qubit)],qubit[(i+1)%len(qubit)]))
            
        circuit.append(cirq.measure(qubit[i%len(qubit)],key=f"qubit_{i}"))
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    for inputs_data in data:
        circuit,qubit=quantum_circuit(inputs_data)
        simulator=cirq.Simulator()
        result=simulator.run(circuit)
        measure_results=[]
        for i in range(len(inputs_data)):
            measure_key=f"qubit_{i}"
            if measure_key in result.measurements:
                measure_results.append(result.measurements[measure_key].flatten()[0])
        results.append(measure_results)
    return np.array(results,dtype=np.float32)
class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,inputs):
            output=tf.numpy_function(run_quantum_circuit,[inputs],tf.float32)
            output.set_shape((inputs.shape[0],inputs.shape[1]))
            return output
model2=Sequential([
    QuantumLayer(),
    
#     Flatten(),
    Dense(32,activation='relu'),
    Dense(64,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)),
    LeakyReLU(0.08),
    Dense(128,kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01)),
    LeakyReLU(0.08),
    Dense(1)
])
model2.compile(optimizer="adam",loss="mse",metrics=["mae"])
model2.summary()
model2.fit(X_train, Y_train, batch_size=32, epochs=10,validation_split=0.2,callbacks=[early_stopping])


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ quantum_layer_20 (QuantumLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 411ms/step - loss: 1606.1624 - mae: 38.9205 - val_loss: 1506.8375 - val_mae: 37.7043
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 366ms/step - loss: 1656.3301 - mae: 39.5385 - val_loss: 1457.7474 - val_mae: 37.0539
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 368ms/step - loss: 1546.2214 - mae: 38.1766 - val_loss: 1408.8158 - val_mae: 36.4068
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 372ms/step - loss: 1479.8676 - mae: 37.4091 - val_loss: 1367.2234 - val_mae: 35.7958
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 361ms/step - loss: 1359.3925 - mae: 35.6947 - val_loss: 1306.5477 - val_mae: 34.8578
Epoch 5: early stopping


In [ ]:
X_train.shape

In [ ]:
Y_train.shape